In [3]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import branca.colormap as cm
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import shape
from shapely import wkt
import numpy as np
import json

In [26]:
census_groups_df = './data/Census_Block_Groups_2010.geojson'

gdf_census = gpd.read_file(census_groups_df)
gdf_census['center'] = gdf_census[['INTPTLAT10', 'INTPTLON10']].agg(','.join, axis=1).str.replace('+', '')
gdf_census['area_m2'] = gdf_census['ALAND10']
gdf_census = gdf_census[['center', 'area_m2', 'geometry']]
gdf_census.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   center    1336 non-null   object  
 1   area_m2   1336 non-null   int32   
 2   geometry  1336 non-null   geometry
dtypes: geometry(1), int32(1), object(1)
memory usage: 26.2+ KB


In [5]:
census_groups_df = './final/census_blocks.geojson'
gdf_census_iso = gpd.read_file(census_groups_df)
gdf_census_iso.head()

,id,INTPTLAT10,INTPTLON10,first_iso_center,geometry
0,"39.96876598164982,-75.1997268790072",39.968758,-75.199725,"39.96876598164982,-75.1997268790072","POLYGON ((-75.19851 39.96945, -75.19744 39.969..."
1,"39.96655556342398,-75.2004472970602",39.966547,-75.200446,"39.96655556342398,-75.2004472970602","POLYGON ((-75.19783 39.96571, -75.20006 39.965..."
2,"39.96430097635668,-75.18964523537966",39.964293,-75.189644,"39.96430097635668,-75.18964523537966","POLYGON ((-75.18766 39.9645, -75.18755 39.9639..."
3,"39.9753665759928,-75.2113493640991",39.975358,-75.211348,"39.9753665759928,-75.2113493640991","POLYGON ((-75.20984 39.97351, -75.21221 39.973..."
4,"39.972428220268384,-75.20517069878659",39.972420,-75.205169,"39.972428220268384,-75.20517069878659","POLYGON ((-75.19855 39.9733, -75.19854 39.9730..."


In [6]:
gdf_census[['lat','lon']] = (
    gdf_census['center']
      .str.split(',', expand=True)
      .astype(float)
)

merged = gdf_census.merge(
    gdf_census_iso[['INTPTLAT10','INTPTLON10','id']],
    left_on = ['lat','lon'],
    right_on= ['INTPTLAT10','INTPTLON10'],
    how     = 'left'
)

merged['center'] = merged['id']

final = merged[['center','area_m2','geometry']]

final.head()


,center,area_m2,geometry
0,"39.96876598164982,-75.1997268790072",161887,"POLYGON ((-75.19851 39.96945, -75.19744 39.969..."
1,"39.96655556342398,-75.2004472970602",103778,"POLYGON ((-75.19783 39.96571, -75.20006 39.965..."
2,"39.96430097635668,-75.18964523537966",43724,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639..."
3,"39.9753665759928,-75.2113493640991",108966,"POLYGON ((-75.20984 39.97351, -75.21221 39.973..."
4,"39.972428220268384,-75.20517069878659",142244,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730..."


In [9]:
gdf_iso = final.copy()

In [10]:
file_path = './data/yelp_academic_dataset_business.json'
df_yelp = pd.read_json(file_path, lines=True)

df_philly = df_yelp[df_yelp['city'] == 'Philadelphia'].copy()
df_philly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14569 entries, 3 to 150336
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   14569 non-null  object 
 1   name          14569 non-null  object 
 2   address       14569 non-null  object 
 3   city          14569 non-null  object 
 4   state         14569 non-null  object 
 5   postal_code   14569 non-null  object 
 6   latitude      14569 non-null  float64
 7   longitude     14569 non-null  float64
 8   stars         14569 non-null  float64
 9   review_count  14569 non-null  int64  
 10  is_open       14569 non-null  int64  
 11  attributes    13399 non-null  object 
 12  categories    14560 non-null  object 
 13  hours         11785 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 1.7+ MB


In [11]:
geometry = [Point(xy) for xy in zip(df_philly['longitude'], df_philly['latitude'])]
gdf_business = gpd.GeoDataFrame(df_philly, geometry=geometry, crs='EPSG:4326')
gdf_joined = gpd.sjoin(gdf_business, gdf_iso, how='inner', predicate='within')
avg_stars = gdf_joined.groupby('index_right')['stars'].mean()
gdf_iso['avg_stars'] = gdf_iso.index.map(avg_stars)
gdf_iso.head(7)

,center,area_m2,geometry,avg_stars
0,"39.96876598164982,-75.1997268790072",161887,"POLYGON ((-75.19851 39.96945, -75.19744 39.969...",NaN
1,"39.96655556342398,-75.2004472970602",103778,"POLYGON ((-75.19783 39.96571, -75.20006 39.965...",NaN
2,"39.96430097635668,-75.18964523537966",43724,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639...",NaN
3,"39.9753665759928,-75.2113493640991",108966,"POLYGON ((-75.20984 39.97351, -75.21221 39.973...",NaN
4,"39.972428220268384,-75.20517069878659",142244,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730...",NaN
5,"39.97434330350556,-75.20735593273575",111388,"POLYGON ((-75.20663 39.97526, -75.20436 39.974...",2.750000
6,"39.97545325216546,-75.22815363105919",147334,"POLYGON ((-75.22648 39.97485, -75.22647 39.974...",2.833333


In [27]:
file_path_choice = './data/ChoiceNeighborhoods.geojson'

gdf_choice = gpd.read_file(file_path_choice)
gdf_choice.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   OBJECTID       4 non-null      int32   
 1   NAME           4 non-null      object  
 2   TYPE           4 non-null      object  
 3   Shape__Area    4 non-null      float64 
 4   Shape__Length  4 non-null      float64 
 5   geometry       4 non-null      geometry
dtypes: float64(2), geometry(1), int32(1), object(2)
memory usage: 308.0+ bytes


In [13]:
if gdf_iso.crs != gdf_choice.crs:
    gdf_choice = gdf_choice.to_crs(gdf_iso.crs)

choice_union = gdf_choice.unary_union

gdf_iso['choice'] = gdf_iso.geometry.intersects(choice_union)
gdf_iso.head()

/var/folders/kz/3nj3xvg96sg9rnf641wm6p_h0000gn/T/ipykernel_28776/2804094208.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  choice_union = gdf_choice.unary_union


,center,area_m2,geometry,avg_stars,choice
0,"39.96876598164982,-75.1997268790072",161887,"POLYGON ((-75.19851 39.96945, -75.19744 39.969...",NaN,True
1,"39.96655556342398,-75.2004472970602",103778,"POLYGON ((-75.19783 39.96571, -75.20006 39.965...",NaN,True
2,"39.96430097635668,-75.18964523537966",43724,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639...",NaN,True
3,"39.9753665759928,-75.2113493640991",108966,"POLYGON ((-75.20984 39.97351, -75.21221 39.973...",NaN,False
4,"39.972428220268384,-75.20517069878659",142244,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730...",NaN,False


In [28]:
file_path_no_thru_use= './data/No_thru_Trucks.geojson'

gdf_no_trucks = gpd.read_file(file_path_no_thru_use)
gdf_no_trucks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2728 entries, 0 to 2727
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   OBJECTID       2728 non-null   int32   
 1   ON_            2728 non-null   object  
 2   FROM_          2728 non-null   object  
 3   TO_            2726 non-null   object  
 4   YEAR           2691 non-null   object  
 5   SEG_ID         2728 non-null   int32   
 6   ST_CODE        2728 non-null   int32   
 7   ONEWAY         2728 non-null   object  
 8   ST_NAME        2728 non-null   object  
 9   STNAME         2728 non-null   object  
 10  Shape__Length  2728 non-null   float64 
 11  geometry       2728 non-null   geometry
dtypes: float64(1), geometry(1), int32(3), object(7)
memory usage: 223.9+ KB


In [15]:
gdf_no_trucks_proj = gdf_no_trucks.to_crs(epsg=4326)

joined = gpd.sjoin(gdf_no_trucks_proj, gdf_iso, how='inner', predicate='intersects')

length_by_block = joined.groupby('index_right')['Shape__Length'].sum()

gdf_iso['no_truck_length'] = gdf_iso.index.map(length_by_block).fillna(0)
gdf_iso.head()

,center,area_m2,geometry,avg_stars,choice,no_truck_length
0,"39.96876598164982,-75.1997268790072",161887,"POLYGON ((-75.19851 39.96945, -75.19744 39.969...",NaN,True,0.000000
1,"39.96655556342398,-75.2004472970602",103778,"POLYGON ((-75.19783 39.96571, -75.20006 39.965...",NaN,True,0.000000
2,"39.96430097635668,-75.18964523537966",43724,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639...",NaN,True,0.000000
3,"39.9753665759928,-75.2113493640991",108966,"POLYGON ((-75.20984 39.97351, -75.21221 39.973...",NaN,False,0.000000
4,"39.972428220268384,-75.20517069878659",142244,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730...",NaN,False,2146.077336


In [29]:
file_path_pool= './data/PPR_Swimming_Pools.geojson'

gdf_swimming_pools = gpd.read_file(file_path_pool)
gdf_swimming_pools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   OBJECTID        72 non-null     int32              
 1   AMENITY_NAME    72 non-null     object             
 2   PARK_NAME       72 non-null     object             
 3   ADDRESS_911     72 non-null     object             
 4   ZIP_CODE        72 non-null     object             
 5   POOL_TYPE       72 non-null     object             
 6   POOL_STATUS     72 non-null     object             
 7   DATE_INSTALLED  22 non-null     datetime64[ms, UTC]
 8   COMMENTS        42 non-null     object             
 9   DATA_SOURCE     72 non-null     object             
 10  geometry        72 non-null     geometry           
dtypes: datetime64[ms, UTC](1), geometry(1), int32(1), object(8)
memory usage: 6.0+ KB


In [17]:
if gdf_iso.crs != gdf_swimming_pools.crs:
    gdf_swimming_pools = gdf_swimming_pools.to_crs(gdf_iso.crs)

gdf_iso['distance_pool'] = gdf_iso.geometry.apply(lambda poly: gdf_swimming_pools.distance(poly).min())
gdf_iso.head(10)

/var/folders/kz/3nj3xvg96sg9rnf641wm6p_h0000gn/T/ipykernel_28776/2713150581.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_iso['distance_pool'] = gdf_iso.geometry.apply(lambda poly: gdf_swimming_pools.distance(poly).min())


,center,area_m2,geometry,avg_stars,choice,no_truck_length,distance_pool
0,"39.96876598164982,-75.1997268790072",161887,"POLYGON ((-75.19851 39.96945, -75.19744 39.969...",NaN,True,0.000000,0.001575
1,"39.96655556342398,-75.2004472970602",103778,"POLYGON ((-75.19783 39.96571, -75.20006 39.965...",NaN,True,0.000000,0.000000
2,"39.96430097635668,-75.18964523537966",43724,"POLYGON ((-75.18766 39.9645, -75.18755 39.9639...",NaN,True,0.000000,0.008896
3,"39.9753665759928,-75.2113493640991",108966,"POLYGON ((-75.20984 39.97351, -75.21221 39.973...",NaN,False,0.000000,0.001422
4,"39.972428220268384,-75.20517069878659",142244,"POLYGON ((-75.19855 39.9733, -75.19854 39.9730...",NaN,False,2146.077336,0.005254
5,"39.97434330350556,-75.20735593273575",111388,"POLYGON ((-75.20663 39.97526, -75.20436 39.974...",2.750000,False,0.000000,0.003170
6,"39.97545325216546,-75.22815363105919",147334,"POLYGON ((-75.22648 39.97485, -75.22647 39.974...",2.833333,False,0.000000,0.001238
7,"39.97164615142378,-75.2209475000264",249138,"POLYGON ((-75.21982 39.97364, -75.21948 39.973...",2.666667,False,506.472963,0.004866
8,"39.972252484959206,-75.22591884039616",86719,"POLYGON ((-75.22261 39.97213, -75.22367 39.971...",3.750000,False,0.000000,0.005179
9,"39.97706012947086,-75.21896321074453",622507,"POLYGON ((-75.21242 39.97399, -75.21221 39.973...",3.062500,False,0.000000,0.001421


In [30]:
rtt_df = './data/rtt_data.csv'
df = pd.read_csv(rtt_df)
df.info()

/var/folders/kz/3nj3xvg96sg9rnf641wm6p_h0000gn/T/ipykernel_28776/922145774.py:2: DtypeWarning: Columns (26,34,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(rtt_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296448 entries, 0 to 296447
Data columns (total 49 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   objectid                      296448 non-null  int64  
 1   document_id                   296448 non-null  int64  
 2   document_type                 296448 non-null  object 
 3   display_date                  296448 non-null  object 
 4   street_address                295592 non-null  object 
 5   zip_code                      289073 non-null  float64
 6   ward                          292480 non-null  float64
 7   grantors                      296371 non-null  object 
 8   grantees                      296408 non-null  object 
 9   cash_consideration            67159 non-null   float64
 10  other_consideration           40168 non-null   float64
 11  total_consideration           61288 non-null   float64
 12  assessed_value                37939 non-null

In [19]:
df['display_date'] = pd.to_datetime(df['display_date'], errors='coerce')

deed_df = df[
    (df['document_type'] == 'DEED') &
    (df['assessed_value'].notnull()) &
    (df['display_date'].dt.year > 2010)
]
deed_df.head(30)

,objectid,document_id,document_type,display_date,street_address,zip_code,ward,grantors,grantees,cash_consideration,...,street_postdir,reg_map_id,matched_regmap,opa_account_num,legal_remarks,discrepancy,property_count,record_id,lat,lng
146804,248538556,52308739,DEED,2011-01-17 05:00:00+00:00,4706-14 WESTMINSTER AVE,19131.0,6.0,BAYVIEW LOAN SERVICING LLC,4706 WESTMINSTER LLC,180000.0,...,NaN,059N240362,059N240405,NaN,NaN,False,1.0,523087399394031,39.968700,-75.216413
146805,248538557,52308740,DEED,2011-01-04 05:00:00+00:00,1648 W MENTOR ST,19141.0,13.0,FEDERAL HOME LOAN MORTGAGE CORPORATION; RITCHI...,KRAPIVIN ALEXANDER,27000.0,...,NaN,135N140083,135N140083,132396400.0,NaN,False,1.0,523087409394032,40.026083,-75.152427
146832,248539016,52309193,DEED,2011-01-17 05:00:00+00:00,2832 D ST,19134.0,7.0,HUNG PHAM TAN; NGUYEN HUNG V,PHAM HUNG TAN,1.0,...,NaN,035N040129,035N040129,71492500.0,NaN,False,1.0,523091939394540,39.992334,-75.122569
146859,248539469,52309631,DEED,2011-01-26 05:00:00+00:00,141 SIGEL ST,19148.0,1.0,DRINKHOUSE LORRAINE; DRINKHOUSE RICHARD,DRINKHOUSE LORRAINE; DRINKHOUSE RICHARD; DRINK...,1.0,...,NaN,014S050124,014S050124,11156800.0,NaN,False,1.0,523096319395044,39.924665,-75.148899
146860,248539470,52309632,DEED,2011-01-26 05:00:00+00:00,145 SIGEL ST,19148.0,1.0,DRINKHOUSE RICHARD,DRINKHOUSE LORRAINE; DRINKHOUSE RICHARD,1.0,...,NaN,014S050126,014S050126,11157000.0,NaN,False,1.0,523096329395045,39.924679,-75.149004
146914,248540360,52310499,DEED,2011-01-18 05:00:00+00:00,4834 PARRISH ST,19139.0,44.0,ROUSE LOUISE E ESTATE OF; ROUSE MARVIN ANTHONY,ROUSE LUCILLE; ROUSE MARVIN ANTHONY,1.0,...,NaN,060N050223,060N050223,441313800.0,NaN,False,1.0,523104999396035,39.967140,-75.218058
147208,248530614,52301249,DEED,2011-01-05 05:00:00+00:00,5245 N 6TH ST,19120.0,49.0,CASILLAS NARCISA ESTATE OF; TORRES MARIA,TORRES NANCY,1.0,...,NaN,123N160102,123N160102,492107600.0,NaN,False,1.0,523012499385233,40.031540,-75.133284
147236,248531061,52301673,DEED,2011-01-03 05:00:00+00:00,2854 N FRANKLIN ST,19133.0,37.0,FLORES MONICA; PINA MILAGROS C,CARTAGENA ALBERTO,65000.0,...,NaN,035N120212,035N120212,372073900.0,NaN,False,1.0,523016739385727,39.995404,-75.144062
147367,248533099,52303648,DEED,2011-01-11 05:00:00+00:00,1457 KERBAUGH ST,19140.0,13.0,COMMISSIONER OF RECORDS; JENKINS DAISY; JENKIN...,MORRIS MARVIN,1.0,...,NaN,101N120276,101N120276,132267400.0,NaN,False,1.0,523036489387986,40.013345,-75.152009
147368,248533100,52303649,DEED,2011-01-11 05:00:00+00:00,3815 PEARL ST,19104.0,24.0,COMMISSIONER OF RECORDS,WILSON ANTHONY; WILSON MICHAEL C,1.0,...,NaN,056N190126,056N190126,241159500.0,NaN,False,1.0,523036499387988,39.960181,-75.197659


In [20]:
gdf_deeds = gpd.GeoDataFrame(
    deed_df,
    geometry=gpd.points_from_xy(deed_df['lng'], deed_df['lat']),
    crs="EPSG:4326"
)
gdf_iso = gdf_iso.to_crs(epsg=4326)
gdf_joined = gpd.sjoin(
    gdf_deeds,
    gdf_iso,
    how='inner',
    predicate='within'
)
median_prices = (
    gdf_joined
      .groupby('index_right')['assessed_value']
      .median()
)
gdf_iso['avg_housing_price'] = gdf_iso.index.map(median_prices)
gdf_iso[['avg_housing_price']].head()

,avg_housing_price
0,3808.0
1,6752.0
2,7824.0
3,5920.0
4,4064.0


In [21]:
foreclosure_df = df.loc[
    df['document_type'].isin(["SHERIFF'S DEED", "DEED SHERIFF"]) &
    df['assessed_value'].notnull() &
    (df['display_date'].dt.year > 2010)
]
foreclosure_df.head(30)

,objectid,document_id,document_type,display_date,street_address,zip_code,ward,grantors,grantees,cash_consideration,...,street_postdir,reg_map_id,matched_regmap,opa_account_num,legal_remarks,discrepancy,property_count,record_id,lat,lng
168705,248579236,52346716,DEED SHERIFF,2011-05-06 04:00:00+00:00,5313 PENTRIDGE ST,19143.0,51.0,MERCURY GROUP INC,SGNK LLC,15100.00,...,NaN,026S200245,026S200245,511127300.0,NaN,False,1.0,523467169439210,39.944541,-75.226786
169246,248587846,52353834,DEED SHERIFF,2011-05-09 04:00:00+00:00,5613 WARRINGTON AVE,19143.0,51.0,BERKMAN MICHAEL,OCONNOR CHRISTIAN; VAN ZANDT W SCOTT,32000.00,...,NaN,026S180364,026S180364,513304400.0,NaN,False,1.0,523538349448782,39.939836,-75.228823
169247,248587847,52353835,DEED SHERIFF,2011-05-09 04:00:00+00:00,1540 S 55TH ST,19143.0,51.0,BERKMAN MICHAEL,OCONNOR CHRISTIAN; VAN ZANDT W SCOTT,32000.00,...,NaN,028S150260,028S150260,514009300.0,NaN,False,1.0,523538359448783,39.937780,-75.223486
169248,248587848,52353836,DEED SHERIFF,2011-05-09 04:00:00+00:00,6014 ANGORA TER,19143.0,3.0,BERKMAN MICHAEL,OCONNOR CHRISTIAN; VAN ZANDT W SCOTT,3400.00,...,NaN,025S150130,025S150130,34030500.0,NaN,False,1.0,523538369448784,39.943351,-75.243149
169249,248587849,52353837,DEED SHERIFF,2011-05-24 04:00:00+00:00,2136 HAWORTH ST,19124.0,62.0,ALTIMAIRO ANN MARIE; BANDURA JOHN M,FEDERAL HOME LOAN MORTGAGE CORPORATION,6400.00,...,NaN,089N160198,089N160198,622024700.0,NaN,False,1.0,523538379448785,40.011219,-75.072308
169465,248576962,52344498,DEED SHERIFF,2011-04-26 04:00:00+00:00,7100 N 19TH ST,19126.0,10.0,TAGGART BOYD,BANK OF NEW YORK MELLON TR,45000.00,...,NaN,145N200296,145N200296,101121400.0,NaN,False,1.0,523444989436687,40.062425,-75.148113
169466,248576963,52344499,DEED SHERIFF,2011-04-20 04:00:00+00:00,1834-58 E WILLARD ST,19134.0,45.0,NEW PIKE CLEANERS INC,CRUZ CARMEN,10500.00,...,NaN,024N140222,NaN,885540340.0,NaN,False,1.0,523444999436688,39.996427,-75.110900
169617,248579249,52346719,DEED SHERIFF,2011-05-06 04:00:00+00:00,1436 W YORK ST,19132.0,16.0,TENNESSEE AVENUE LLC,PBB PROPERTY HOLDINGS LLC,16123.73,...,NaN,030N200315,030N200315,161273100.0,NaN,False,6.0,523467199439225,39.989376,-75.156982
170157,248587850,52353838,DEED SHERIFF,2011-05-24 04:00:00+00:00,6634 OAKLAND ST,19149.0,54.0,MCCUTCHEON MINGO III; MCGILL TONYA,US BANK NATIONAL ASSOCIATION TR,6900.00,...,NaN,139N100244,139N100244,541257800.0,LEGAL DESCRITPION DISCREPANCY FRONTAGE DIMENSI...,False,1.0,523538389448786,40.040067,-75.069411
170158,248587851,52353839,DEED SHERIFF,2011-05-24 04:00:00+00:00,7807 FAYETTE ST,19150.0,50.0,HARRIS KEITH B,EMC MORTGAGE CORPORATION,20000.00,...,NaN,143N240391,143N240391,501030800.0,NaN,False,1.0,523538399448788,40.071716,-75.163520


In [22]:
foreclosure_df = foreclosure_df.copy()
gdf_fore = gpd.GeoDataFrame(
    foreclosure_df,
    geometry=gpd.points_from_xy(foreclosure_df['lng'], foreclosure_df['lat']),
    crs="EPSG:4326"
)
gdf_joined = gpd.sjoin(
    gdf_fore,
    gdf_iso,
    how='inner',
    predicate='within'
)
counts = gdf_joined.groupby('index_right').size()
gdf_iso['foreclosure_count'] = (
    gdf_iso.index.map(counts)
              .fillna(0)
              .astype(int)
)
gdf_iso[['foreclosure_count']].head()

,foreclosure_count
0,0
1,4
2,0
3,2
4,3


In [23]:
gdf_iso['foreclosure_over_area'] = (gdf_iso['foreclosure_count'] / gdf_iso['area_m2'])

In [24]:
gdf_iso_features_final = gdf_iso.copy()
gdf_iso_features_final = gdf_iso_features_final.drop(columns=['geometry'])
gdf_iso_features_final.head()

,center,area_m2,avg_stars,choice,no_truck_length,distance_pool,avg_housing_price,foreclosure_count,foreclosure_over_area
0,"39.96876598164982,-75.1997268790072",161887,NaN,True,0.000000,0.001575,3808.0,0,0.000000
1,"39.96655556342398,-75.2004472970602",103778,NaN,True,0.000000,0.000000,6752.0,4,0.000039
2,"39.96430097635668,-75.18964523537966",43724,NaN,True,0.000000,0.008896,7824.0,0,0.000000
3,"39.9753665759928,-75.2113493640991",108966,NaN,False,0.000000,0.001422,5920.0,2,0.000018
4,"39.972428220268384,-75.20517069878659",142244,NaN,False,2146.077336,0.005254,4064.0,3,0.000021


In [25]:
gdf_iso_features_final.to_csv("./final/census_features.csv")